# Play-space for `intake_dataframe_catalog`

In [1]:
%cd /g/data/tm70/ds0092/software/intake-dataframe-catalog

/g/data/tm70/ds0092/software/intake-dataframe-catalog


In [2]:
import intake

## Create a DF Catalog using `add`

In [3]:
from intake_dataframe_catalog.core import DFCatalogModel

In [4]:
dfcat = DFCatalogModel(metadata_columns=["meta"])

In [5]:
cat = intake.open_csv("test.csv")

In [6]:
for i in range(3):
    cat.name = f"csv{i}"
    if i == 0:
        dfcat.add(cat, metadata={"meta": "['0','1']"})
    else:
        dfcat.add(cat, metadata={"meta": f"['{i}']"})

In [7]:
dfcat.save("dfcat")

In [8]:
dfcat.df

,name,meta,yaml
0,csv0,"['0','1']",sources:\n csv0:\n args:\n urlpath: t...
1,csv1,['1'],sources:\n csv1:\n args:\n urlpath: t...
2,csv2,['2'],sources:\n csv2:\n args:\n urlpath: t...


## Create a DF Catalog directly from a dict

In [9]:
from collections import defaultdict
entries = defaultdict(list)
for i in range(3):
    cat.name = f"csv{i}"
    entries["cat"].append(cat)
    entries["meta"].append(i)

In [10]:
dfcat = DFCatalogModel.from_dict(entries, cat_key="cat")

## Use the DF Catalog

In [11]:
import ast

dfcat = intake.open_df_catalog("dfcat.csv", read_kwargs={"converters": {"meta": ast.literal_eval}})

In [12]:
dfcat.search(meta=["0","1"]).df

,name,meta,yaml
0,csv0,"[0, 1]",sources:\n csv0:\n args:\n urlpath: t...
1,csv1,[1],sources:\n csv1:\n args:\n urlpath: t...


In [13]:
dfcat.search(meta=["0","1"], require_all=True).df

,name,meta,yaml
0,csv0,"[0, 1]",sources:\n csv0:\n args:\n urlpath: t...


In [14]:
dfcat["csv0"].read()

,first_name,last_name
0,joe,blogs
1,jane,doe


In [15]:
dfcat["csv0"](csv_kwargs={"include_path_column":True}).read()

,first_name,last_name,path
0,joe,blogs,/g/data/tm70/ds0092/software/intake-dataframe-...
1,jane,doe,/g/data/tm70/ds0092/software/intake-dataframe-...


In [16]:
dfcat.serialize("dfcat2")

## Write to a yaml

In [17]:
dfcat.name = "dfcat"
yaml_text = dfcat.yaml()
print(yaml_text)
with open("./dfcat.yaml", mode="wt") as f:
    f.write(yaml_text)

sources:
  dfcat:
    args:
      path: dfcat.csv
      read_kwargs:
        converters:
          meta: !!python/name:ast.literal_eval ''
    description: ''
    driver: intake_dataframe_catalog.core.DFFileCatalog
    metadata: {}



In [18]:
cat = intake.open_catalog("dfcat.yaml")

ConstructorError: could not determine a constructor for the tag 'tag:yaml.org,2002:python/name:ast.literal_eval'
  in "<unicode string>", line 7, column 17:
              meta: !!python/name:ast.literal_eval ''
                    ^

In [ ]:
cat.dfcat.df